In [1]:
import os
import pandas as pd
import numpy as np
os.chdir('C:/PhD/thesis/chapter3/abm')

import pickle
import ast

### create simulation summary statistic

In [2]:
age_denom=365
num_of_steps=4000

In [3]:
#load simulation pickle files
targetfolder='base'
with open(f'./{targetfolder}/stats.pkl', 'rb') as f:
    stats_dict=pickle.load(f)

with open(f'./{targetfolder}/lhs.pkl', 'rb') as f:
    lhs=pickle.load(f)

with open(f'./{targetfolder}/dft1_m.pkl', 'rb') as f:
    dft1_map_dict=pickle.load(f)
    
with open(f'./{targetfolder}/dft2_m.pkl', 'rb') as f:
    dft2_map_dict=pickle.load(f)

In [4]:
#cases where dft2 spreads out and dft1 dominates
dd={}
for j in stats_dict:
    dd[j]=4000-stats_dict[j]['dft2 infected'].count(0)
target=[i for i in dd if dd[i]>1]
counter=0
for i in target:
    if max(stats_dict[i]['dft1 infected'])>max(stats_dict[i]['dft2 infected']):
        counter+=1

print(len(target))
print(counter/len(target))
print(max([max(stats_dict[i]['dft1 infected']) for i in target]),max([max(stats_dict[i]['dft2 infected']) for i in target]))

4961
0.9913323926627696
46 29


In [14]:
#cases where dft2 persists
dd={}
for j in stats_dict:
    dd[j]=(stats_dict[j]['dft1 infected'][-1],stats_dict[j]['dft2 infected'][-1])
target1=[i for i in dd if dd[i][0]>0]
target2=[i for i in dd if dd[i][1]>0]
print(len(target1),len(target2))
if len(target2)!=0:
    print(np.mean([dd[j][1] for j in target2]),np.max([dd[j][0] for j in target2]),np.max([dd[j][1] for j in target2]))

2747 5
1.6 7 3


In [6]:
#check if dft1 reach the south and dft2 reach the north
dd={}
for j in stats_dict:
    d1=[k for k in stats_dict[j]['dft1 boundary'] if k!=None]
    d2=[k for k in stats_dict[j]['dft2 boundary'] if k!=None]
    if len(d2)>0:
        dd[j]=(max(d1),min(d2))
    else:
        dd[j]=(max(d1),None)
        
target1=[i for i in dd if dd[i][0]==37]
target2=[i for i in dd if dd[i][1]==0]
print(len(target1)/5000,len(target2)/5000)

0.0726 0.0126


In [11]:
#cases where coinfection exists
dd={}
for j in stats_dict:
    if max(stats_dict[j]['coinfection'])>0:
        dd[j]=(max(stats_dict[j]['coinfection']),stats_dict[j]['coinfection'].count(1))
        
print(len(dd))
print(max([i[0] for i in dd.values()]))
print(max([i[1] for i in dd.values()]))

39
2
96


In [ ]:
#get simulation summary metrics
valid=pd.read_csv('disease.csv')

#baseyear
baseyear=2013

#etl
valid['timestep']=valid['year']-baseyear
valid['id']=valid['id'].apply(lambda x:ast.literal_eval(x))
                  
ssm={}
for i in valid.index:
    if valid.at[i,'timestep'] in ssm:
        ssm[valid.at[i,'timestep']].append(valid.at[i,'id'])
    else:
        ssm[valid.at[i,'timestep']]=[valid.at[i,'id']]

In [24]:
#compute time stamp
rows=[timestep for timestep in range(num_of_steps) if timestep//age_denom in ssm ]

#calc annual average
dft1_row=[]
for k in dft1_map_dict:
    dft1_row.append([j for i in range(0,len(rows),age_denom) for j in np.mean(dft1_map_dict[k][i:i+age_denom],axis=0)])

dft2_row=[]
for k in dft2_map_dict:
    dft2_row.append([j for i in range(0,len(rows),age_denom) for j in np.mean(dft2_map_dict[k][i:i+age_denom],axis=0)])

#create dataframe
arr=np.c_[np.array(dft1_row),np.array(dft2_row)]

spatio=pd.DataFrame(arr)
spatio.columns=[f's{i}' for i in range(1,len(spatio.columns)+1)]

In [25]:
#get total alive and infected
total_alive=[stats_dict[i]['total alive'] for i in stats_dict]
dft1_infected=[stats_dict[i]['dft1 infected'] for i in stats_dict]
dft2_infected=[stats_dict[i]['dft2 infected'] for i in stats_dict]

In [26]:
#take annual average
timegaps=[i for i in range(age_denom,num_of_steps,age_denom)]+[num_of_steps]
ta=[]
i1=[]
i2=[]
for i in range(len(total_alive)):
    ta.append([np.mean(total_alive[i][:j]) for j in timegaps][1:])
    i1.append([np.max(dft1_infected[i][:j]) for j in timegaps][1:])
    i2.append([np.max(dft2_infected[i][:j]) for j in timegaps][1:])

In [27]:
#get lhs params
simulation_parameters=pd.DataFrame().from_dict(lhs)

In [28]:
#get simulation data
da=pd.DataFrame(ta,columns=[f'a{i}' for i in range(1,len(timegaps))])
di1=pd.DataFrame(i1,columns=[f'1d{i}' for i in range(1,len(timegaps))])
di2=pd.DataFrame(i2,columns=[f'2d{i}' for i in range(1,len(timegaps))])
dr1=pd.DataFrame(np.array(i1)/np.array(ta),columns=[f'1r{i}' for i in range(1,len(timegaps))])
dr2=pd.DataFrame(np.array(i2)/np.array(ta),columns=[f'2r{i}' for i in range(1,len(timegaps))])
simulation_summary_metrics=pd.concat([da,di1,di2,dr1,dr2,spatio],axis=1)

In [29]:
target=['a1', 'a2','a5', 'a6', 'a7', 'a8', 'a9', 'a10',
        '1d1', '1d2','1r1', '1r2', '1r5', '1r6', '1r7', '1r8', '1r9', '1r10',  '2r5', '2r6', '2r7', '2r8', '2r9', '2r10']+[f's{i}' for i in range(1,len(spatio.columns)+1)]

In [30]:
simulation_parameters.to_csv(f'./{targetfolder}/simulation_parameters.csv',index=False)
simulation_summary_metrics[target].to_csv(f'./{targetfolder}/simulation_summary_metrics.csv',index=False)

### prepare for rsm

In [14]:
sp=pd.read_csv(f'./{targetfolder}/simulation_parameters.csv')
ssm=pd.read_csv(f'./{targetfolder}/simulation_summary_metrics.csv')

In [15]:
# sp=sp[sp['dft1_intro']>sp['dft2_intro']]

# ssm=ssm.loc[sp.index]

In [16]:
#target from chapter 2
d1_p=[251*2, 250*2]
d1_i=[2*2, 7*2]
d2_p=[105,101,108,61,75,46]

d1_r=[0.15,
  0.17391304347826086,
  0.0,
  0.37037037037037035,
  0.21739130434782608,
  0.5714285714285714]

d2_r=[0.2,
  0.043478260869565216,
  0.42105263157894735,
  0.2222222222222222,
  0.13043478260869565,
  0.0]



In [17]:
#omt concat
omt=d1_p+d2_p+d1_i+np.divide(d1_i,d1_p).tolist()+d1_r+d2_r+valid['DFT1'].tolist()+valid['DFT2'].tolist()

#matching ssm shape
mapping=np.repeat(np.array(omt).reshape(1,-1),len(ssm),axis=0)

#avoid divided by zero
denom=mapping.copy()
denom[denom==0]=0.0000000000000001

#get sum of squared error
sse=(((mapping-np.array(ssm))/denom)**2).sum(axis=1)

In [18]:
sp.loc[np.argsort(sse)[-10:]].mean()

min_resource                  200.700000
max_resource                  523.700000
max_fitness                    14.500000
min_fitness                     5.100000
max_dft1_transmission_rate      0.679408
max_dft1_latency_period       335.200000
min_dft1_latency_period       153.900000
dft1_fitness_decline_rate       0.502208
dft1_fitness_decline_val        2.700000
max_dft2_transmission_rate      0.555216
max_dft2_latency_period       387.500000
min_dft2_latency_period       149.100000
dft2_fitness_decline_rate       0.611392
dft2_fitness_decline_val        2.100000
dft1_intro                    611.000000
dft2_intro                    658.400000
dtype: float64

### random forest

In [19]:
from sklearn.ensemble import RandomForestRegressor

clf=RandomForestRegressor(n_estimators=1000,max_depth=500,oob_score=True,)

X=sp
y=sse

clf.fit(X,y)

print(clf.oob_score_,clf.score(X,y))

print(sorted(zip(X.columns, clf.feature_importances_),key=lambda x:x[1],reverse=True))

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\_array_api.py:390: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


-0.0901737137134404 0.8531653715734202
[('dft2_fitness_decline_rate', 0.13945042705561386), ('max_dft1_transmission_rate', 0.1132123706511548), ('dft1_intro', 0.09786280019126223), ('dft1_fitness_decline_rate', 0.08288887287593451), ('dft2_intro', 0.07912132123628213), ('max_dft2_transmission_rate', 0.07288004670550222), ('min_resource', 0.06406871130169461), ('max_resource', 0.05823360832978168), ('min_dft1_latency_period', 0.05804136797841942), ('max_dft2_latency_period', 0.057336035028456314), ('min_dft2_latency_period', 0.051429052724918406), ('max_dft1_latency_period', 0.04487796601469182), ('max_fitness', 0.0274167682143666), ('min_fitness', 0.02497687120124282), ('dft1_fitness_decline_val', 0.014444328182734898), ('dft2_fitness_decline_val', 0.013759452307943612)]


In [20]:
sadsad

NameError: name 'sadsad' is not defined

In [ ]:
# df=pd.read_csv('brute_force.csv')
# df.columns=X.columns
# result=clf.predict(df)
# dd=pd.DataFrame(result)
# df.loc[dd.sort_values(0).index[:20]]

### quadratic

In [ ]:
# #prep for quadratic function
# num_cols=len(sp.columns)
# for i in range(num_cols):
#     for j in range(i,num_cols):
#         sp[sp.columns[i]+'*'+sp.columns[j]]=sp[sp.columns[i]]*sp[sp.columns[j]]

# #Quadratic Central Composite Design
# from sklearn.linear_model import LinearRegression
# X = sp
# y = sse
# reg = LinearRegression().fit(X, y)

# coeffs=[reg.intercept_]+reg.coef_.tolist()

# reg.score(X,y)

# #initial guess
# init=[300,
#  600.0,
#  20.0,
#  10.0,
#  0.7,
#  300,
#  100,
#  0.3,
#  1.0,
#  0.7,
#  300,
#  100,
#  0.3,
#  1.0,   ]#1095,365,

# #bounds for each variable
# regions=[(200,300),(400,600),(15,20),(7,10),(0.5,0.9),(200,540),(100,180),(0.1,0.9),(1,5),(0.5,0.9),(200,540),(100,180),(0.1,0.9),(1,5),
#         ]#(365,1095),(365,1095)

# # Optimization (Gradient Descent - Simplified)
# from scipy.optimize import minimize

# def obj_func(params):
    
#     #prep for quadratic func
#     arr=[]
#     for i in range(len(params)):
#         for j in range(i,len(params)):
#             arr.append(params[i]*params[j])
    
#     #separate beta
#     beta0=coeffs[0]
#     betas=coeffs[1:]
    
#     cost=beta0+np.r_[np.array(params),np.array(arr)]@(np.array(betas).T)

#     return cost


# result = minimize(obj_func, init, bounds=regions, method='L-BFGS-B',tol=1e-6,
#     options={
#         "maxiter": 10000,  # maximum iterations
#     }) #L-BFGS-B Nelder-Mead
# optimal_params = result.x

# dict(zip(sp.columns[:len(init)],result.x))

### svm

In [ ]:
# import numpy as np
# from itertools import product
# import pandas as pd

# lhs=[]
# lhs.append(np.arange(200,310,100))
# lhs.append(np.arange(400,610,100))
# lhs.append(np.arange(15,21,2))
# lhs.append(np.arange(7,11,2))
# lhs.append(np.arange(0.5,1,0.1))
# lhs.append(np.arange(180,550,150))
# lhs.append(np.arange(90,190,60))
# lhs.append(np.arange(0.1,1,0.4))
# lhs.append(np.arange(1,4))
# lhs.append(np.arange(0.5,1,0.1))
# lhs.append(np.arange(180,550,150))
# lhs.append(np.arange(90,190,60))
# lhs.append(np.arange(0.1,1,0.4))
# lhs.append(np.arange(1,4))


# # Use itertools.product to get the Cartesian product (all combinations)
# combinations = list(product(*lhs))

# # Convert the list of combinations to a numpy array
# combinations_array = np.array(combinations)

# df=pd.DataFrame(combinations_array,columns=range(len(lhs)))
# df.to_csv('brute_force.csv',index=False)


# lhs.append(np.arange(365,1096,365))
# lhs.append(np.arange(365,1096,365))

# # Use itertools.product to get the Cartesian product (all combinations)
# combinations = list(product(*lhs))

# # Convert the list of combinations to a numpy array
# combinations_array = np.array(combinations)

# df=pd.DataFrame(combinations_array,columns=range(len(lhs)))
# df.to_csv('brute_force_time.csv',index=False)

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

In [ ]:
# Scale the data (SVR is sensitive to the scale of the features)
sc_X = StandardScaler()
sc_y = StandardScaler()
X=sp
y=sse

X_scaled = sc_X.fit_transform(X)
y_scaled = sc_y.fit_transform(y.reshape(-1, 1)).ravel()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3)

# Create and train the SVR model
svr = SVR(kernel='rbf', C=100, epsilon=0.0001)
svr.fit(X_train, y_train)
svr.score(X_test, y_test)

In [ ]:
# Create and train the SVR model
svr = SVR(kernel='rbf', C=100, epsilon=0.1)
svr.fit(X_scaled, y_scaled)

In [ ]:
svr.score(X_scaled, y_scaled)

In [ ]:
if 'time' in targetfolder:
    
    df=pd.read_csv('brute_force_time.csv')
else:
    df=pd.read_csv('brute_force.csv')

df.columns=X.columns

In [ ]:
result=svr.predict(sc_X.fit_transform(df))

In [ ]:
dd=pd.DataFrame(result)

df.loc[dd.sort_values(0).index[:100]].mean()

In [ ]:
with open(f'./{targetfolder}/svr.pkl', "wb") as f:
    pickle.dump(svr, f)
    
dd.to_csv(f'./{targetfolder}/result.csv',index=False)